In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from keras.preprocessing import image
import numpy as np
#from google.colab.patches import cv2_imshow
import os
import sys
from PIL import Image

Using TensorFlow backend.


In [11]:
import numpy as np
from keras.preprocessing import image
font = cv2.FONT_HERSHEY_SIMPLEX
#face_cascade = cv2.CascadeClassifier("C:\\Users\\kamma\\Anaconda3\\envs\\vdpnLeP35\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml")
#face_cascade = cv2.CascadeClassifier("C:\\Users\\kamma\\Anaconda3\\envs\\vdpnLeP35\\Library\\etc\\haarcascades\\haarcascade_frontalface_alt_tree.xml")
face_cascade = cv2.CascadeClassifier("C:\\Users\Allworx User 2.ALLWORXW2\\Anaconda3\\envs\\tf15\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml")
#face_cascade = cv2.CascadeClassifier("C:\\Users\\kamma\\Anaconda3\\envs\\vdpnLeP35\\Library\\etc\\haarcascades\\haarcascade_frontalface_alt2.xml")
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
# load the model
model = VGG16()
cap = cv2.VideoCapture(0)
cap.set(5, 30)
frameRate = cap.get(5)
print(frameRate)
count=0
while True:
    #current frame number--this is for frame rate
    count+=1
    frameId=count
    ret, img = cap.read()
    #frameId = cap.get(1)
    #show the mark on face
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    #print(frameId)
    #print(frameRate)
    if (frameId % (math.floor((frameRate/5))) == 0):
        print("Inside if ") 
        #converting ndarray to image
        filename ="vijay.jpg"
        cv2.imwrite(filename, img) 

        # load an image from file
        image = load_img('vijay.jpg', target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # predict the probability across all output classes
        yhat = model.predict(image)
        # convert the probabilities to class labels
        label = decode_predictions(yhat)
        # retrieve the most likely result, e.g. highest probability
        label = label[0][0]
        # print the classification
        print('%s (%.2f%%)' % (label[1], label[2]*100))
        #Write text on the rectanguler box.
        cv2.putText(img, str(label), (286, 104), font, 0.8, (0, 0, 255), 3, cv2.LINE_AA)     
        cv2.imshow('img',img) 
    k= cv2.waitKey(5) & 0xff
    if k==27:
        break
        
cv2.destroyAllWindows()
cap.release()


#https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_table_of_contents_gui/py_table_of_contents_gui.html#py-table-of-content-gui

30.00003000003
Inside if 
sunglasses (19.45%)
Inside if 
sunglasses (15.95%)
Inside if 
sunglasses (14.11%)
Inside if 
sunglasses (16.09%)
Inside if 
sunglasses (16.15%)
Inside if 
sunglasses (18.89%)
Inside if 
sunglasses (13.33%)
Inside if 
sunglasses (17.49%)
Inside if 
sunglasses (20.17%)
Inside if 
sunglasses (17.06%)
Inside if 
sunglasses (18.21%)
Inside if 
sunglasses (19.06%)
Inside if 
sunglasses (13.97%)
Inside if 
lab_coat (11.69%)
Inside if 
sunglasses (11.62%)


In [2]:
import collections
import numpy as np
import itertools

SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

def generate_ssd_priors(specs, image_size=300, clip=True):
    """Generate SSD Prior Boxes.
    
    Args:
        specs: Specs about the shapes of sizes of prior boxes. i.e.
            specs = [
                Spec(38, 8, SSDBoxSizes(30, 60), [2]),
                Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
                Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
                Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
                Spec(3, 100, SSDBoxSizes(213, 264), [2]),
                Spec(1, 300, SSDBoxSizes(264, 315), [2])
            ]
        image_size: image size.
    
    Returns:
        priors: a list of priors: [[center_x, center_y, h, w]]. All the values
            are relative to the image size (300x300).
    """
    boxes = []
    for spec in specs:
        scale = image_size / spec.shrinkage
        for j, i in itertools.product(range(spec.feature_map_size), repeat=2):
            x_center = (i + 0.5) / scale
            y_center = (j + 0.5) / scale

            # small sized square box
            size = spec.box_sizes.min
            h = w = size / image_size
            boxes.append([
                x_center,
                y_center,
                h,
                w
            ])
            
            # big sized square box
            size = np.sqrt(spec.box_sizes.max * spec.box_sizes.min)
            h = w = size / image_size
            boxes.append([
                x_center,
                y_center,
                h,
                w
            ])           
            
            # change h/w ratio of the small sized box
            # based on the SSD implementation, it only applies ratio to the smallest size.
            # it looks wierd.
            size = spec.box_sizes.min
            h = w = size / image_size
            for ratio in spec.aspect_ratios:
                ratio = sqrt(ratio)                  
                boxes.append([
                    x_center,
                    y_center,
                    h * ratio,
                    w / ratio
                ])
                boxes.append([
                    x_center,
                    y_center,
                    h / ratio,
                    w * ratio
                ])
            


    boxes = np.array(boxes)
    if clip:
        boxes = np.clip(boxes, 0.0, 1.0)
    return boxes